In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

def load_data(file_path):
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return X, y

def logistic_regression_reg(X, y, learning_rate, epochs, batch_size, v, gamma_0, d):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0

    for epoch in range(epochs):
        learning_rate_t = gamma_0 / (1 + gamma_0 / d * epoch)
        indices = np.random.permutation(n_samples)
        X_shuffled, y_shuffled = X[indices], y[indices]

        for start in range(0, n_samples, batch_size):
            end = min(start + batch_size, n_samples)
            X_batch, y_batch = X_shuffled[start:end], y_shuffled[start:end]

            linear_model = np.dot(X_batch, weights)
            y_predicted = sigmoid(linear_model)

            gradient = np.dot(X_batch.T, (y_predicted - y_batch)) / batch_size + weights / v
            weights -= learning_rate_t * gradient

    return weights,bias

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def predict(X, weights, biases):
    linear_model = np.dot(X, weights) + biases
    y_predicted = sigmoid(linear_model)
    y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
    return np.array(y_predicted_cls)

def evaluate(X, y, weights, bias):
    linear_model = np.dot(X, weights) + bias
    y_predicted = sigmoid(linear_model)
    predictions = np.round(y_predicted).astype(int)
    return accuracy_score(y, predictions)

X_train, y_train = load_data('bank-note/train.csv')
X_test, y_test = load_data('bank-note/test.csv')

gamma_0 = 0.01
d = 0.1
epochs = 100
batch_size = 64
variances = [0.01, 0.1, 0.5, 1, 3, 5, 10, 100]

for v in variances:
    weights, bias = logistic_regression_reg(X_train, y_train, gamma_0, epochs, batch_size, v, gamma_0, d)
    train_accuracy = evaluate(X_train, y_train, weights, bias)
    test_accuracy = evaluate(X_test, y_test, weights, bias)
    print(f"Variance: {v}, Training Accuracy: {train_accuracy}, Test Accuracy: {test_accuracy}")


Variance: 0.01, Training Accuracy: 0.7052752293577982, Test Accuracy: 0.702
Variance: 0.1, Training Accuracy: 0.7786697247706422, Test Accuracy: 0.788
Variance: 0.5, Training Accuracy: 0.9002293577981652, Test Accuracy: 0.896
Variance: 1, Training Accuracy: 0.930045871559633, Test Accuracy: 0.908
Variance: 3, Training Accuracy: 0.944954128440367, Test Accuracy: 0.926
Variance: 5, Training Accuracy: 0.9438073394495413, Test Accuracy: 0.93
Variance: 10, Training Accuracy: 0.9518348623853211, Test Accuracy: 0.932
Variance: 100, Training Accuracy: 0.9541284403669725, Test Accuracy: 0.936


In [32]:
def compute_gradient_ml(X, y, weights):
    predictions = sigmoid(np.dot(X, weights))
    logistic_grad = np.dot(X.T, (predictions - y)) / len(X)
    return logistic_grad

def train_logistic_regression_ml(X_train, y_train, gamma_0, d, epochs=100):
    weights = np.zeros(X_train.shape[1])
    for epoch in range(epochs):
        permutation = np.random.permutation(len(X_train))
        X_train_shuffled = X_train[permutation]
        y_train_shuffled = y_train[permutation]
        for t, (x, y) in enumerate(zip(X_train_shuffled, y_train_shuffled)):
            gamma_t = gamma_0 / (1 + (gamma_0 / d) * t)
            gradient = compute_gradient_ml(x.reshape(1, -1), y, weights)
            weights -= gamma_t * gradient
    return weights

def compute_error(X, y, weights):
    predictions = predict(X, weights)
    errors = np.mean(predictions != y)
    return errors

def predict(X, weights):
    linear_model = np.dot(X, weights)
    y_predicted = sigmoid(linear_model)
    return np.round(y_predicted).astype(int)

gamma_0 = .01
d = 0.1

weights_ml = train_logistic_regression_ml(X_train, y_train, gamma_0, d)

train_error_ml = compute_error(X_train, y_train, weights_ml)
test_error_ml = compute_error(X_test, y_test, weights_ml)

print(f"ML Estimation - Training Error: {train_error_ml * 100:.2f}%, Test Error: {test_error_ml * 100:.2f}%")

ML Estimation - Training Error: 4.01%, Test Error: 4.80%
